In [3]:
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [4]:
from matplotlib import cm
from scipy.optimize import curve_fit
from scipy.signal import argrelextrema

In [5]:
matplotlib.rcParams['figure.figsize'] = [10.5,7]
matplotlib.rcParams.update({'font.size': 20})

In [6]:
# pas top pour avoir cov avec barre erreur
def lin_fit(y1,y2,deg):
    p=polyfit(y1,y2,deg) # obtention coefficients fit
    return poly1d(p) # génération fonction

In [5]:
# # Mathieu Barre erreur
# # on peut lui donner barre erreur pour calcul variance
# def function_(x,a,b) :
#     return cos(a*x+b)*rand(1)*x
# x=arange(0,10,0.1)
# A = rand(1)
# B=0.2*rand(1)
# to_fit = function_(x,A,B) # pts xp
# fit_ , pcov = curve_fit(function_,x,to_fit,p0=[A,B]) # le fit
# ERR_fit = np.sqrt(np.diag(pcov)) # 1sigma variance
# figure('function_').clf()
# plt.plot(x,to_fit,label='data')
# plt.errorbar(x,function_(x,fit_[0],fit_[1]),yerr=ERR_fit[0],label='FIT')
# plt.legend()

# Extraction de données

In [6]:
filename=[]

In [1]:
# sélection des fichiers (Python 3)
import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
root.withdraw()

file_path = filedialog.askopenfilename(initialdir = '/home/adrian/Documents/Mesures',multiple=True)
print(file_path)
# /home/adrian/Documents/Mesures/20180503/

('/home/adrian/Documents/Mesures/GiantMol_Ions/20180503/tickle_120-8vpp_2500mv_167k_367k_000.dat', '/home/adrian/Documents/Mesures/GiantMol_Ions/20180503/tickle_129-8vpp_2500mv_167k_367k_000.dat', '/home/adrian/Documents/Mesures/GiantMol_Ions/20180503/tickle_129-9vpp_2500mv_167k_367k_000.dat', '/home/adrian/Documents/Mesures/GiantMol_Ions/20180503/tickle_139-8vpp_2500mv_167k_367k_000.dat', '/home/adrian/Documents/Mesures/GiantMol_Ions/20180503/tickle_149-7vpp_2500mv_167k_367k_000.dat', '/home/adrian/Documents/Mesures/GiantMol_Ions/20180503/tickle_160-3vpp_2500mv_167k_367k_000.dat', '/home/adrian/Documents/Mesures/GiantMol_Ions/20180503/tickle_170-3vpp_2500mv_167k_367k_000.dat', '/home/adrian/Documents/Mesures/GiantMol_Ions/20180503/tickle_180-3vpp_2500mv_167k_367k_000.dat', '/home/adrian/Documents/Mesures/GiantMol_Ions/20180503/tickle_190-3vpp_2500mv_167k_367k_001.dat')


In [7]:
filename=file_path
# Temps, PM, Tension Pzt bleu
TP = zeros((len(filename),999))
PM = zeros((len(filename),999),int)
VB = zeros((len(filename),999))
for k in range(0,len(filename),1):
    TP[:][k],PM[:][k],VB[:][k] = loadtxt('%s' %filename[k],usecols=(0,1,3),unpack=True)  

In [8]:
for k in range(0,len(file_path)):
    print(file_path[k][40:62])
print(file_path[1][63:72])

Ions/20180503/tickle_1
Ions/20180503/tickle_1
Ions/20180503/tickle_1
Ions/20180503/tickle_1
Ions/20180503/tickle_1
Ions/20180503/tickle_1
Ions/20180503/tickle_1
Ions/20180503/tickle_1
Ions/20180503/tickle_1
9-8vpp_25


In [9]:
# plot tickle
cm = pylab.get_cmap('jet')
# cmap=cm(1.*to_plot/len(file_path))

freq=linspace(167,367,len(PM[0]))
test_min=[]

figure('Plot tickle').clf()
rows=[multiply(asarray(argrelextrema(PM[k],less,order=50)),0.2)+167 for k in range(0,len(file_path))]

for k in range(0,len(file_path)):
    toto=[]
    for l in range(0,shape(rows[k])[1]):
        toto.append(rows[k].item(l))
    test_min.append(toto)
print('test_min = ',test_min)

# test_min=[[rows[lin].item(col) for lin in range(0,len(file_path))] for col in enumerate(rows(col))]
# find minimum
# for k in range(0,len(file_path)):
#     test_min[k,]=multiply(argrelextrema(PM[k],less,order=50),0.2)+167
#     print('row',row)
# #     test_min.append(row)
#     row[k][:]=[row[k,x] for x in range(0,8)]
#     plot(freq[0:len(PM[k])],PM[k],color = cm(1.*k/len(file_path)))
#     plt.vlines(x=row,ymin=0,ymax=80000,linestyle=':',color = cm(1.*k/len(file_path)))
    
# test_min = np.array(list(test_min))
    
xlabel('Frequency (kHz)')
ylabel('Fluorescence (counts)')
grid()
plt.tight_layout()

test_min =  [[167.2, 181.2, 205.4, 253.0, 293.2, 330.6, 350.6, 366.0], [167.8, 185.2, 201.0, 223.2, 260.2, 280.8, 309.20000000000005, 321.8, 348.4], [171.6, 201.0, 223.2, 260.8, 284.4, 321.0, 350.20000000000005, 364.4], [192.2, 204.8, 223.0, 243.2, 262.2, 307.8, 324.0, 343.6, 364.8], [171.4, 187.6, 230.4, 244.2, 263.4, 280.8, 302.8, 325.4, 347.4, 366.4], [167.2, 211.6, 226.2, 262.2, 283.8, 317.0, 333.0, 353.8], [178.8, 199.6, 217.8, 262.4, 288.2, 303.8, 327.6, 349.8, 365.6], [170.8, 184.4, 217.2, 235.0, 261.8, 279.4, 290.2, 309.8, 325.0, 355.0], [167.4, 211.4, 223.6, 243.60000000000002, 262.8, 301.4, 331.6, 345.8]]


In [14]:
test_min =  [[167.2, 181.2, 205.4, 253.0, 293.2, 330.6, 350.6, 366.0], [167.8, 185.2, 201.0, 223.2, 260.2, 280.8, 309.20000000000005, 321.8, 348.4], [192.2, 204.8, 223.0, 243.2, 262.2, 307.8, 324.0, 343.6, 364.8], [171.4, 187.6, 230.4, 244.2, 263.4, 280.8, 302.8, 325.4, 347.4, 366.4], [167.2, 211.6, 226.2, 262.2, 283.8, 317.0, 333.0, 353.8], [178.8, 199.6, 217.8, 262.4, 288.2, 303.8, 327.6, 349.8, 365.6], [170.8, 184.4, 217.2, 235.0, 261.8, 279.4, 290.2, 309.8, 325.0, 355.0], [167.4, 211.4, 223.6, 243.60000000000002, 262.8, 301.4, 331.6, 345.8]]
freq_min = [multiply(test_min[k],0.2)+167 for k in range(0,len(file_path))]
freq_min

IndexError: list index out of range

In [14]:
# true_min=[]
# for k in range(0,len(file_path)):
#     temp_min=[]
#     for l in range(0,3):
#         idx = argmin(asarray(test_min[0])[:])
#         temp_min.append(idx)
#         delete(test_min,idx)
#     true_min.append(temp_min)
# print(true_min)

to_plot = 3
figure('Plot tickle seul').clf()
plot(freq[0:len(PM[to_plot])],PM[to_plot],color = cm(1.*to_plot/len(file_path)))
plt.vlines(x=test_min[to_plot],ymin = 0,ymax = 80000,linestyle = ':',color = cm(1.*k/len(file_path)))
xlabel('Frequency (kHz)')
ylabel('Fluorescence (counts)')
grid()
plt.tight_layout()

In [13]:
test_min

[[167.2, 181.2, 205.4, 253.0, 293.2, 330.6, 350.6, 366.0],
 [167.8, 185.2, 201.0, 223.2, 260.2, 280.8, 309.20000000000005, 321.8, 348.4],
 [171.6, 201.0, 223.2, 260.8, 284.4, 321.0, 350.20000000000005, 364.4],
 [192.2, 204.8, 223.0, 243.2, 262.2, 307.8, 324.0, 343.6, 364.8],
 [171.4, 187.6, 230.4, 244.2, 263.4, 280.8, 302.8, 325.4, 347.4, 366.4],
 [167.2, 211.6, 226.2, 262.2, 283.8, 317.0, 333.0, 353.8],
 [178.8, 199.6, 217.8, 262.4, 288.2, 303.8, 327.6, 349.8, 365.6],
 [170.8, 184.4, 217.2, 235.0, 261.8, 279.4, 290.2, 309.8, 325.0, 355.0],
 [167.4, 211.4, 223.6, 243.60000000000002, 262.8, 301.4, 331.6, 345.8]]

In [12]:
to_delete=[[0,4,5,6,7],
[0,1,5,6,7,8,9],
[0,1,5,6,7,8,9],
[0,1,2,6,7,8,9],
[0,1,2,5,6,7],
[0,1,2,6,7,8],
[0,1,2,3,5,6,9],
[0,1,2,3,5]]

true_test_min=[]
for k in range(0,len(file_path)):
    true_test_min.append(list(delete(test_min[k],to_delete[k])))

true_test_min[3]=[244.2,259,263.4]
true_test_min[4]=[262.2,267,283.8]

true_test_min

/home/adrian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  if sys.path[0] == '':


IndexError: list index out of range

In [387]:
# plot one curve

to_plot=7
figure('Plot tickle seul').clf()
plot(freq[0:len(PM[to_plot])],PM[to_plot],color = cm(1.*to_plot/len(file_path)))
plt.vlines(x=true_test_min[to_plot],ymin=0,ymax=80000,linestyle=':',color = cm(1.*k/len(file_path)))
xlabel('Frequency (kHz)')
ylabel('Fluorescence (counts)')
grid()
plt.tight_layout()

In [ ]:
# choose only good minimum

# test_min[to_plot] = delete(test_min[to_plot],(0,1,2,3,5))
# figure('Plot tickle seul').clf()
# plot(freq[0:len(PM[to_plot])],PM[to_plot],color = cm(1.*to_plot/len(file_path)))
# plt.vlines(x=test_min[to_plot],ymin=0,ymax=80000,linestyle=':',color = cm(1.*k/len(file_path)))
# xlabel('Frequency (kHz)')
# ylabel('Fluorescence (counts)')
# grid()
# plt.tight_layout()

In [11]:
save_min=true_test_min
lz=[2,2,2,2,0,0,0,0]
wz=[]
for k in range(0,len(file_path)):
    wz.append(save_min[k][lz[k]]/2)
print(wz)
save_min=np.column_stack((save_min,wz))
save_min

NameError: name 'true_test_min' is not defined

In [10]:
URF=divide([120.8,129.9,139.8,147.7,160.3,170.3,180.3,190.3],4)
lines=[[0,0,0,0,1,1,1,1], # 30 175
       [1,1,1,1,2,2,2,2], # 30 200
       [2,2,2,2,0,0,0,0], # 30 250 2wz
       [3,3,3,3,3,3,3,3]] # wz
lines=asmatrix(lines)

cm2 = pylab.get_cmap('brg')
span=linspace(29,48,1e4)

fig=figure('tickle minimum')
ax = fig.add_subplot(1, 1, 1)

dy=[1,1,5,2.5]
for k in range(0,len(file_path)): # URF
    for l in range(0,4):
#         plot(URF[k],asmatrix(save_min[k]),'+',color=cm(1.*k/len(file_path)),mew=2,ms=8,label='_nolegend_')
        errorbar(URF[k],asmatrix(save_min[k,lines[l,k]]),yerr=dy[l],fmt='x',mew=1,ms=6,color='black',ecolor=cm2(1.*l/4),elinewidth=2,capsize=5,label='_nolegend_')
#         plot(URF,wz,'+',color=cm(1.*k/len(file_path)),mew=2,ms=8)
#         toto[k,l]=save_min.item(k,lines.item(l,k))
#         print(save_min[k],URF[k])

# plt.errorbar(URF, y, yerr=dy, fmt='o', color='black',
#              ecolor='lightgray', elinewidth=3, capsize=0);
    
R=[[],[],[],[]]
lstyle=['--', '--', ':', ':','-', '--', '-.', ':']
for l in range(0,4):
    lefit=[]
    for k in range(0,len(file_path)):
        lefit.append(save_min[k].item(lines.item(l,k)))
    poly_fit=lin_fit(URF,lefit,1)
    plot(span,poly_fit(span),linestyle=lstyle[l],color=cm2(1.*l/4))
    print(poly_fit)
    R[l]=corrcoef(URF,lefit)

# font = {'family': 'sans-serif',
#         'weight': 'normal',
#         'size': 20,
#         }
# props = dict(facecolor='white', alpha=1)
# xpos=[43.5,43.5,43.5,43.5]
# ypos=[282,340,247,145]
# for k in range(0,4,1):
#     plt.text(xpos[k],ypos[k],'R = {:.4f}'.format(R[k].item(0,1)),
#              fontdict=font,color=cm2(1.*k/4),bbox=props)

ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=2.5))
ax.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=25)) 
    
legend(['$\omega_x$','$\omega_y$','$2\omega_z$','$\omega_z$'])
    
xlabel('$U_{RF}$')
ylabel('Tickle frequency response (kHz)')
grid()
tight_layout()

/home/adrian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  if __name__ == '__main__':


NameError: name 'save_min' is not defined

In [392]:
e=1.602e-19
m=40.078*1.66054e-27 # 40.078

r0=2.5e-3 # 2.5e-3
d0=4e-3/2

Omega=2.047e6*2*pi # 2.047e6

In [393]:
Udc=0
kappa=0.22481324
zeta=kappa*r0**2/d0**2
Vend=2.5

In [401]:
dw=r0**(-1)*sqrt((e/m)*(Udc-zeta*Vend+0.5*e/(m*r0**2*Omega**2)*URF**2))
print('dw')
for k in range(0,len(dw)):
    print('{:.1f}'.format(dw[k]/1000/(2*pi)),'kHz')
    
dw2=Omega*0.5*sqrt(4*e/(m*r0**2*Omega**2)*(Udc-zeta*Vend)+2*(e/(m*r0**2*Omega**2))**2*URF**2)
print('dw2')
for k in range(0,len(dw2)):
    print('{:.1f}'.format(dw2[k]/1000/(2*pi)),'kHz')

    
ax=4*e/(m*r0**2*Omega**2)*(Udc-Vend)
q=2*e*URF/(m*r0**2*Omega**2)
dw3=Omega*0.5*sqrt(ax+q**2/2)
print('dw2')
for k in range(0,len(dw2)):
    print('{:.1f}'.format(dw3[k]/1000/(2*pi)),'kHz')

dw
42.3 kHz
58.4 kHz
72.8 kHz
83.2 kHz
98.3 kHz
109.6 kHz
120.4 kHz
130.9 kHz
dw2
42.3 kHz
58.4 kHz
72.8 kHz
83.2 kHz
98.3 kHz
109.6 kHz
120.4 kHz
130.9 kHz
dw2
nan kHz
nan kHz
nan kHz
nan kHz
nan kHz
nan kHz
nan kHz
36.3 kHz


/home/adrian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in sqrt
  


In [397]:
w=[]
for k in range(0,len(file_path)): # URF
    temp=[]
    for l in range(0,4):
#         plot(URF[k],asmatrix(save_min[k]),'+',color=cm(1.*k/len(file_path)),mew=2,ms=8,label='_nolegend_')
        temp.append(save_min[k,lines[l,k]])
    w.append(temp)
w=asmatrix(w)
dwmes=zeros(8)
for k in range(0,8):
    dwmes[k]=w[k,1]-w[k,0]
print('dwmes',dwmes,'kHz')

dwmes [24.2 22.2 20.2 14.8 16.8 15.6 15.2 14.2] kHz


In [396]:
-zeta*Vend,0.5*e/(m*r0**2*Omega**2)*URF

(-0.87817671875,
 array([0.03515674, 0.03780514, 0.04068636, 0.04298552, 0.04665253,
        0.04956286, 0.05247319, 0.05538351]))